In [34]:
import sys
import os

pwd = os.getcwd()
python_path = pwd[: pwd.rfind("/")]
sys.path.append(python_path)

In [35]:
import torch
import torch.nn as nn
from torch.optim import lr_scheduler
from models.pggan import Generator
from lightning.pytorch.callbacks import ModelSummary

In [36]:
def conv2d_size_out(size: int, kernel_size: int, padding: int = 0, dilation: int = 1, stride: int = 1) -> int:
    return (size + 2 * padding - dilation * (kernel_size - 1) - 1) // stride + 1 

def conv_transpose_2d_out(size: int, kernel_size: int, stride: int = 1, padding: int = 0, output_padding: int = 0, dilation: int = 1) -> int:
    return (size - 1) * stride - 2 * padding + dilation * (kernel_size - 1) + output_padding + 1

In [37]:
x = torch.randn((1, 28, 28))

conv2d_args = {"kernel_size": 4, "stride": 3}
conv_transpose_2d_args = {"kernel_size": 4, "stride": 3}

print(f"""
Predict:
conv2d_size_out = {conv2d_size_out(size=28, **conv2d_args)}
conv_transpose_2d_out = {conv_transpose_2d_out(size=28, **conv_transpose_2d_args)}

Real:
conv2d_size_out = {nn.Conv2d(1, 16, **conv2d_args)(x).shape}
conv_transpose_2d_out = {nn.ConvTranspose2d(1, 100, **conv_transpose_2d_args)(x).shape}
""")


Predict:
conv2d_size_out = 9
conv_transpose_2d_out = 85

Real:
conv2d_size_out = torch.Size([16, 9, 9])
conv_transpose_2d_out = torch.Size([100, 85, 85])



### Discriminator: 28x28 -> 16x16 -> 8x8 -> 4x4 -> 1
### Generator: 4x4 -> 7x7 -> 14x14 -> 28x28

In [38]:
s = conv_transpose_2d_out(size=4, kernel_size=4, stride=1)
print(s)
s = conv_transpose_2d_out(size=s, kernel_size=4, stride=2, padding=1)
print(s)
s = conv_transpose_2d_out(size=s, kernel_size=4, stride=2, padding=1)
print(s)

7
14
28


In [39]:
def block(in_channels: int, out_channels: int, kernel_size: int, stride: int, padding: int):
    return [
        nn.ConvTranspose2d(
            in_channels,
            out_channels,
            kernel_size=kernel_size,
            stride=stride,
            padding=padding,
            bias=False,
        ),
        nn.BatchNorm2d(out_channels),
        nn.ReLU(True),
    ]

model = nn.Sequential(
    *block(in_channels=10, out_channels=64, kernel_size=4, stride=1, padding=0),
    *block(in_channels=64, out_channels=64, kernel_size=8, stride=2, padding=1),
    nn.ConvTranspose2d(
        in_channels=64, out_channels=1, kernel_size=4, stride=2, padding=1
    ),
    nn.Tanh()
)

model(torch.randn(20, 10, 1, 1)).shape


torch.Size([20, 1, 24, 24])

In [40]:
def block(in_channel: int, out_channels: int, norm: bool = True):
    layers: List[Any] = [
        nn.Conv2d(
            in_channels=in_channel,
            out_channels=out_channels,
            kernel_size=(4, 4),
            stride=2,
            padding=1,
            bias=False,
        ),
    ]
    if norm:
        layers.append(nn.BatchNorm2d(out_channels, affine=True))
        layers.append(nn.LeakyReLU(0.2, inplace=True))
    return layers

model = nn.Sequential(
    *block(1, 64, norm=False),
    *block(64, 128),
    # *block(128, 256),
    *block(128, 256),
    nn.Conv2d(256, 1, kernel_size=7, stride=2, bias=False),
)

y = model(torch.randn(100, 1, 64, 64))
y.shape

torch.Size([100, 1, 1, 1])

In [41]:
optim = torch.optim.Adam([torch.rand((10, ))], lr=0.001)
scheduler = lr_scheduler.ConstantLR(optim, factor=0.9, total_iters=10)
for i in range(10):
    scheduler.step()

/opt/conda/envs/venv/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:139: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


In [42]:
optim = torch.optim.Adam([torch.rand((10, ))], lr=0.001)
scheduler = lr_scheduler.LinearLR(optim, start_factor=1, end_factor=0.1, total_iters=10)
for i in range(20):
    if i > 5:
        scheduler.step()
    print(scheduler.get_lr())

[0.001]
[0.001]
[0.001]
[0.001]
[0.001]
[0.001]
[0.0008281]
[0.0007389010989010989]
[0.0006498780487804878]
[0.0005610958904109588]
[0.00047265624999999993]
[0.0003847272727272727]
[0.00029760869565217393]
[0.00021189189189189193]
[0.00012892857142857145]
[5.2631578947368444e-05]
[0.00010000000000000003]
[0.00010000000000000003]
[0.00010000000000000003]
[0.00010000000000000003]


/opt/conda/envs/venv/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:558: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


In [43]:
class NN(nn.Module):
    def __init__(self, *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)
        
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        m.weight.data.normal_(0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)

In [44]:
model = nn.Sequential()
model.append(nn.BatchNorm2d(2))
model.append(nn.Linear(10, 10))
model.add_module('hee', nn.Linear(1, 4))
model

Sequential(
  (0): BatchNorm2d(2, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (1): Linear(in_features=10, out_features=10, bias=True)
  (hee): Linear(in_features=1, out_features=4, bias=True)
)

In [45]:
def deepcopy_module(module):
    new_module = nn.Sequential()
    for _, module in module.named_children():
        new_module.append(module)
        new_module[-1].load_state_dict(module.state_dict())        
    return new_module

In [46]:
new_mod = deepcopy_module(model)
new_mod.append(nn.Linear(1, 1))

Sequential(
  (0): BatchNorm2d(2, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (1): Linear(in_features=10, out_features=10, bias=True)
  (2): Linear(in_features=1, out_features=4, bias=True)
  (3): Linear(in_features=1, out_features=1, bias=True)
)

In [49]:
G = Generator()
G(torch.FloatTensor(512, 4, 1, 1))

RuntimeError: Calculated padded input size per channel: (1 x 1). Kernel size: (3 x 3). Kernel size can't be greater than actual input size